In [2]:
import os
import pymssql

import pandas as pd

from dotenv import load_dotenv

load_dotenv()

def get_connection(db='processo'):
    return pymssql.connect(
        server=os.getenv("SQL_SERVER_HOST"),
        user=os.getenv("SQL_SERVER_USER"),
        password=os.getenv("SQL_SERVER_PASS"),
        port=os.getenv("SQL_SERVER_PORT"),
        database=db
    )



from langchain_openai import AzureOpenAI, AzureChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate, FewShotChatMessagePromptTemplate

llm = AzureChatOpenAI(model_name="gpt-4o")


In [4]:
sql_decisoes = """ 
SELECT CONCAT(numero_processo, '/', ano_processo) as processo, * 
FROM processo.dbo.Ata_Composicao_Pautas
WHERE ( year(DataInclusao) = 2024 AND month(DataInclusao) IN (10, 11, 12) )
"""
df_decisoes_20244 = pd.read_sql(sql_decisoes, get_connection())

sql_decisoes = """ 
SELECT CONCAT(numero_processo, '/', ano_processo) as processo, * 
FROM processo.dbo.Ata_Composicao_Pautas
WHERE ( year(DataInclusao) = 2025 AND month(DataInclusao) IN (1, 2, 3) )
"""
df_decisoes_20251 = pd.read_sql(sql_decisoes, get_connection())


/tmp/ipykernel_9637/3780874636.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_decisoes_20244 = pd.read_sql(sql_decisoes, get_connection())
/tmp/ipykernel_9637/3780874636.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_decisoes_20251 = pd.read_sql(sql_decisoes, get_connection())


In [5]:
len(df_decisoes_20244), len(df_decisoes_20251)

(979, 956)

In [ ]:
df_decisoes_20244.columns

Index(['processo', 'IdComposicaoPauta', 'codigo_camara', 'numero_sessao',
       'ano_sessao', 'numero_processo', 'ano_processo', 'ordem_informacao',
       'setor', 'resumo', 'parecer_proc', 'voto', 'interessado', 'assunto',
       'usuario', 'data_ultima_atualizacao', 'acordaos', 'Pro_saiu_num_sessao',
       'Pro_saiu_ano_sessao', 'Pro_remanescente_num_sessao',
       'Pro_remanescente_ano_sessao', 'Relator', 'Relator_substituido',
       'titulo_detalhe', 'texto_detalhe', 'ordemInformacaoProc', 'setorProc',
       'ordemInformacaoVoto', 'setorVoto', 'resumoVoto', 'resultadoTipo',
       'numeroResultado', 'anoResultado', 'resultado', 'ementa',
       'arquivoFisico', 'cabecalhoAcordao', 'rodapeAcordao', 'idInformacao',
       'idSessao', 'dataInclusao', 'usuarioInclusao',
       'processoLiberadoPublicacao', 'dataGeracaoArquivoPublicacao',
       'usuarioGeracaoArquivoPublicacao', 'DataPublicacaoDOE',
       'UsuarioInclusaoPublicacaoDOE', 'PaginaDOE', 'ValorMulta',
       'ValorRe

In [6]:
df_decisoes = df_decisoes_20244[~df_decisoes_20244['resultado'].isna()]

In [7]:
df_decisoes_20244[['processo']]

,processo
0,002074/2020
1,001879/2020
2,019423/2001
3,102167/2019
4,002742/2017
...,...
974,704436/2013
975,707525/2019
976,019033/2017
977,701447/2013


In [ ]:
examples = [
      {
          "input": """
          - Adoção das providências cabíveis no tocante aos indícios de impropriedades /irregularidades
  elencadas na tabela 19 do Relatório (transcritas acima ), com fundamento no art. 2º, inciso III, da
  Resolução nº 012/2023-TCE, notadamente para que a Secretaria de Controle Externo analise a
  capacidade operacional desta Corte com vistas ao acompanhamento e /ou à abertura de processo
  autônomo, respeitadas a conveniência e oportunidade, referente aos itens 4.2 (Divergência dos
  dados do planejamento do Projeto, Inconsistência dos saldos da conta bens móveis entre os
  registros do SIGEF - Sistema Integrado de Planejamento e Gestão Fiscal - e SMI - Sistema de
  Monitoramento e Informações do Projeto - e deficiências na elaboração das notas explicativas ),
  5.5.1.1 (Inventário físico ), 5.6.1.1 ( Falhas/deficiências construtivas do Hospital Regional da
  Mulher Parteira Maria Correia) e 5.6.1.3 (Pendências de legalização das obras da Biblioteca
  Câmara Cascudo e da Sede do Serviço Nacional de Emprego – SINE).""",
          "output": "OBRIGACAO"}, {
          "input": """( a.i) promovam, no prazo de 120 (cento e vinte dias) úteis, contados a partir da intimação da
  presente Decisão, a apuração dos fatos e se verifique a constitucionalidade e legalidade dos
  vínculos funcionais de cada servidor que figura nos Anexos nºs. 01 e 02 contidos nos Eventos
  nºs. 04 e 05, além de outros que porventura sejam informados pela DDP em cumprimento ao
  item b, por meio da instauração de processos administrativos disciplinares individuais ,
  regulados pela Lei que trata do Estatuto Jurídico dos Servidores do respectivo Município ,
  com observância dos princípios do contraditório, ampla defesa e devido processo legal;
    ( a.ii) comprovem neste feito, em 05 dias úteis após ultimado o prazo de definição dos PAD
  ´s, as conclusões de todos os processos administrativos instaurados, no tocante à eliminação
  de tríplice vínculo funcional identificado e de enquadramento das eventuais acumulações
  dúplices nas hipóteses permitidas pela Constituição Federal, com a respectiva
  compatibilidade de horários, sob pena de, não cumprindo tais obrigações nos prazos antes
  referidos, incidir em multa diária e pessoal ao gestor, no valor de R$ 500,00, com espeque no
  art. 110 da LCE nº 464/2012 c/c o art. 326 do RITCE, cabendo à Diretoria de Despesa com
  Pessoal monitorar o cumprimento da presente Decisão;""",
          "output": "OBRIGACAO"}, {
          "input": """Vistos, relatados e discutidos estes autos, em consonância ao posicionamento do
  Corpo técnico e do Ministério Público de Contas, ACORDAM os Conselheiros, nos termos
  do voto proposto pelo Conselheiro Relator, julgar a inadmissibilidade da presente denúncia e
  o seu conseqüente arquivamento, com fulcro nos art. 12 do Provimento 002/2020 –
  CORREG/TCE, aprovado pela Resolução 016/2020 – TCE e artigo 80, § 1º, da LOTCE.
  E ainda, Pela expedição de RECOMENDAÇÃO, nos termos do art. 301, III da Resolução
  009/2012 (RITCE/RN) c/c art. 13, II da Resolução 16/2020 –TCE/RN, ao Executivo
  Municipal de Nísia Floresta /RN, com cópia para o respectivo órgão de controle interno, ou
  setor responsável pelas publicações oficiais, lastreada na Constituição Federal de 88/ Art. 37,
  a fim de que promova e deixe claro os seguintes comportamentos em suas postagens""",
          "output": "OBRIGACAO"}, {
          "input": """
Ante o exposto, discordando do Corpo Técnico da DAM e do Ministério Público de Contas – que se manifestaram no sentido de julgar o mérito do feito pela aprovação da matéria – VOTO no sentido de:
1) preliminarmente, considero inexistentes os documentos constantes no processo apensado n. 300799/2024, na forma do art. 166, § 2º, do RITCE;
2) rejeitar a preliminar de incompetência desta Corte de Contas para a análise das contratações objeto dos autos;
3) nos termos dos arts. 120 e 121 da LCE nº 464/2012, conceder, de ofício, tutela provisória (medida cautelar) para determinar aos atuais gestores dos Municípios de Lagoa Nova/RN, Cruzeta/RN e Jucurutu/RN, que:
a) tão logo sejam intimados, suspendam de forma imediata os pagamentos de honorários advocatícios em favor de Fernanda de Paula Sociedade Individual de Advocacia, decorrentes dos Contratos objeto dos autos, ainda que encerrada a sua vigência, ou de termos aditivos destes, acaso existentes, até o julgamento de mérito do presente feito, sob pena de multa pessoal ao gestor recalcitrante no valor de R$ 50.000,00 (cinquenta mil reais) por cada novo pagamento realizado;
b) assim que forem intimados, promovam, também de forma imediata e por meio de ato formal, a habilitação das Procuradorias de cada um dos Municípios nos autos dos processos administrativos e/ou judiciais nos quais Fernanda de Paula Sociedade Individual de Advocacia representa os Entes municipais;
c) no prazo de 10 (dez) dias úteis, a contar de sua intimação, comprovem nestes autos o exato cumprimento das medidas cautelares ora determinadas, especialmente no que tange à suspensão imediata dos pagamentos e à habilitação, também imediata, das Procuradorias nos processos que Fernanda de Paula Sociedade Individual de Advocacia representa os respectivos Entes municipais. No mesmo prazo, devem juntar ao presente feito eventuais termos aditivos realizados com Fernanda de Paula Sociedade Individual de Advocacia, ou comprovar que não houve, sob pena de multa pessoal ao gestor recalcitrante no valor de R$ 5.000,00 (cinco mil reais) por cada dia de eventual descumprimento destas ordens, na forma do art. 110, da LCE nº 464/2012;
d) com base no art. 82, inciso V, da Lei Complementar Estadual n. 464/12, e no art. 288, do Regimento Interno do TCE/RN, pela determinação do MONITORAMENTO do cumprimento da presente Decisão, por intermédio da DAM;
4) a citação dos Srs. Iogo Nielson de Queiroz e Silva, Luciano Silva Santos e Joaquim José de Medeiros, Prefeitos Municipais de Jucurutu/RN, Lagoa Nova/RN e Cruzeta/RN, respectivamente, além de Fernanda de Paula Sociedade Individual de Advocacia, com arrimo nos artigos 37, c/c 45, §1º, inciso II, ambos da Lei Complementar Estadual nº 464/2012, a fim de que, querendo, apresentem suas razões de defesa;
5) a remessa imediata de cópia da decisão ora proferida ao Ministério Público Comum Estadual, haja vista a probabilidade de dano ao erário em razão dos pagamentos já realizados e que seguem acontecendo, possibilitando-lhe o conhecimento para eventuais providências no âmbito de sua competência.
Intimem-se da forma mais célere possível, nos termos do art. 46, § 4º da LCE 464/2012, além dos atuais gestores dos Municípios de Lagoa Nova/RN, Cruzeta/RN e Jucurutu/RN, também a Representada Fernanda de Paula Sociedade Individual de Advocacia.
Sala das Sessões, data da assinatura eletrônica.

(assinado digitalmente)
Carlos Thompson Costa Fernandes
Conselheiro Relator
""",
          "output": "OBRIGACAO"}, {
          "input": """
          Pelo exposto, concordando parcialmente com o Representante e em dissonância com o Ministério Público de Contas – divirjo do Representante em relação ao prazo proposto e deste último por entender pelo deferimento do pedido cautelar e não pela citação da gestora –, VOTO:

a) pelo acolhimento em parte do pedido cautelar apontado no item “b” da Exordial, ao passo que determino a intimação do Município de Tenente Ananias/RN e de sua gestora para que:

(a.i) promovam, no prazo de 120 (cento e vinte dias) úteis, contados a partir da intimação da presente Decisão, a apuração dos fatos e se verifique a constitucionalidade e legalidade dos vínculos funcionais de cada servidor que figura na Informação Técnica da DDP junto ao evento nº 33, por meio da instauração de processos administrativos disciplinares individuais, regulados pela Lei que trata do Estatuto Jurídico dos Servidores do respectivo Município, com observância dos princípios do contraditório, ampla defesa e devido processo legal;

(a.ii) comprovem neste feito, em 05 dias úteis após ultimado o prazo de definição dos PAD´s, as conclusões de todos os processos administrativos instaurados, no tocante à eliminação de tríplice vínculo funcional identificado e de enquadramento das eventuais acumulações dúplices nas hipóteses permitidas pela Constituição Federal, com a respectiva compatibilidade de horários, sob pena de, não cumprindo tais obrigações nos prazos antes referidos, incidir em multa diária e pessoal à gestora, no valor de R$ 500,00, com espeque no art. 110 da LCE nº 464/2012 c/c o art. 326 do RITCE, cabendo à Diretoria de Despesa com Pessoal monitorar o cumprimento da presente Decisão;

b) com base no art. 82, inciso V, da Lei Complementar Estadual nº 464/12, e no art. 288 do Regimento Interno do TCE/RN, pela determinação do MONITORAMENTO do cumprimento da presente Decisão, por intermédio da DDP.


(assinado digitalmente)
Carlos Thompson Costa Fernandes
Conselheiro Relator
          """,
          "output": "OBRIGACAO"},{
              "input": """Vistos, relatados e discutidos estes autos, acolhendo os fundamentos do parecer
  ministerial, com substrato no art. 209 V da norma regimental, ACORDAM os Conselheiros ,
  nos termos do voto proposto pela Conselheira Relatora, julgar pelo ARQUIVAMENTO dos
  autos.""",
              "output": "OUTROS",
          },
          {
              "input": """Vistos, relatados e discutidos estes autos, ACORDAM os Conselheiros,  com o
  impedimento do Conselheiro Presidente Renato Costa Dias, nos termos do voto profposto
  pela Conselheira Relatora, haja vista os fundamentos fático -jurídicos explanados no excerto
  antecedente, comprovado documentalmente o adimplemento substancial do plano de
  redimensionamento/adequação do sistema de ensino natalense, julgar pela EXTINÇÃO do
  FEITO nos termos do art. 71 da Lei Complementar (estadual) c/c art. 22 §1° da LINDB e art .
  209 V da regra regimental.""",
              "output": "OUTROS",
          },
          {
              "input": """Vistos, relatados e discutidos estes autos, em consonância com o posicionamento da
  Diretoria de Administração Municipal – DAM e do Ministério Público de Contas ,
  ACORDAM os Conselheiros, nos termos do voto proferido pelo Conselheiro Relator, julgar
  pelo reconhecimento da incidência da Prescrição Intercorrente sobre a pretensão punitiva e
  ressarcitória desta Corte de Contas, nos termos do artigo 111, parágrafo único, da Lei
  Complementar Estadual nº 464/2012, com o consequente arquivamento dos presentes autos.
  E ainda, pelo envio de cópia das principais peças dos autos ao Ministério Público Estadual ,
  para conhecimento e atuação no âmbito de sua competência.""",
              "output": "OUTROS",
          },
          {
              "input": """Vistos, relatados e discutidos estes autos, em dissonância com o Ministério Público
  de Contas, ACORDAM os Conselheiros, nos termos do voto proferido pelo Conselheiro
  Relator, julgar pela extinção do feito, sem julgamento de mérito, com o consequente
  ARQUIV AMENTO dos autos, em virtude da incompatibilidade entre a emissão de parecer
  prévio pela aprovação com ressalvas das contas de governo e a instauração de apuração de
  responsabilidade para aplicação de sanções.""",
              "output": "OUTROS",
          }
    ]

example_prompt = ChatPromptTemplate.from_messages([
        ("human", '''
         {input} '''),
        ("ai", "{output}"),
    ])

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """Você é um classificador de decisões de um tribunal de contas.
         Sua tarefa é definir se uma decisão trata de uma multa ou obrigação
         de fazer.

         Responda com DETERMINACAO se o texto contiver o termo "multa" e alguma recomendação ou obrigação de fazer
         Responda com OUTROS se o texto tratar de arquivamento, extinção do feito ou prescrição da matéria, ou outro assunto que não seja DETERMINACAO

         Responda APENAS com DETERMINACAO ou OUTROS."""),
        few_shot_prompt,
        ("human", "{input}"),])


In [ ]:
chain = final_prompt | llm
decision_type = chain.invoke({"input": state["messages"]}).content

In [14]:
df_decisoes_20244['resultado'].fillna('', inplace=True)

In [19]:
for r in df_decisoes_20244[df_decisoes_20244['resultado'].str.contains('promov')]['resultado']:
    print(r)
    print('---')

Ante o exposto, discordando do Corpo Técnico da DAM e do Ministério Público de Contas – que se manifestaram no sentido de julgar o mérito do feito pela aprovação da matéria – VOTO no sentido de:
1) preliminarmente, considero inexistentes os documentos constantes no processo apensado n. 300799/2024, na forma do art. 166, § 2º, do RITCE;
2) rejeitar a preliminar de incompetência desta Corte de Contas para a análise das contratações objeto dos autos;
3) nos termos dos arts. 120 e 121 da LCE nº 464/2012, conceder, de ofício, tutela provisória (medida cautelar) para determinar aos atuais gestores dos Municípios de Lagoa Nova/RN, Cruzeta/RN e Jucurutu/RN, que:
a) tão logo sejam intimados, suspendam de forma imediata os pagamentos de honorários advocatícios em favor de Fernanda de Paula Sociedade Individual de Advocacia, decorrentes dos Contratos objeto dos autos, ainda que encerrada a sua vigência, ou de termos aditivos destes, acaso existentes, até o julgamento de mérito do presente feito, 

In [24]:
for r in df_decisoes_20244[df_decisoes_20244['resultado'].str.contains('obrig')]['resultado']:
    print(r)
    print('---')

30.	Diante do exposto, concordando com o Corpo Técnico e com o Parecer do Ministério Público junto a esta Corte, proponho aos Excelentíssimos Conselheiros integrantes desta primeira Câmara de Contas o julgamento pela DESAPROVAÇÃO DA MATÉRIA, nos termos do artigo 75, II, da Lei Complementar Estadual nº 464/2012, sem prejuízo da aplicação de multa ao Sr. Rosemberg Monteiro de Carvalho, responsável à época pela Câmara Municipal de Severiano Melo, no valor de R$ 5.000,00 (cinco mil reais), com fulcro no artigo 107, inciso II, “f”, da Lei Complementar nº 464/2012 c/c o artigo 33, inciso I, alínea “c”, da Resolução nº 011/2016 – TCE/RN, tendo em vista a omissão total na alimentação de informações no Portal da Transparência do referido ente, relativas ao período fiscalizado (exercício de 2019), com violação de disposição regulamentar acima especificada. 
31.	Proponho, também, a expedição de determinação ao atual gestor responsável pela Câmara Municipal de Severiano Melo/RN para que providenci

In [20]:
print(df_decisoes_20244[df_decisoes_20244['resultado'].str.contains('realiz')].iloc[0]['resultado'])

30.	Diante do exposto, concordando com o Corpo Técnico e com o Parecer do Ministério Público junto a esta Corte, proponho aos Excelentíssimos Conselheiros integrantes desta primeira Câmara de Contas o julgamento pela DESAPROVAÇÃO DA MATÉRIA, nos termos do artigo 75, II, da Lei Complementar Estadual nº 464/2012, sem prejuízo da aplicação de multa ao Sr. Rosemberg Monteiro de Carvalho, responsável à época pela Câmara Municipal de Severiano Melo, no valor de R$ 5.000,00 (cinco mil reais), com fulcro no artigo 107, inciso II, “f”, da Lei Complementar nº 464/2012 c/c o artigo 33, inciso I, alínea “c”, da Resolução nº 011/2016 – TCE/RN, tendo em vista a omissão total na alimentação de informações no Portal da Transparência do referido ente, relativas ao período fiscalizado (exercício de 2019), com violação de disposição regulamentar acima especificada. 
31.	Proponho, também, a expedição de determinação ao atual gestor responsável pela Câmara Municipal de Severiano Melo/RN para que providenci